In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
import re

path = '/Users/zhengao/Documents/TechMining'

In [ ]:
a = '/Data/PAs_test100.csv'
DATA = pd.read_csv(path + a)

In [ ]:
# 检查空值
# DATA[DATA.isnull().values == True]

In [ ]:
del DATA['_TTL'], DATA['_ABST'], DATA['_ISD']
DATA['_ACLM'][43] = '\n 8.'
DATA['_ACLM'][47] = '\n 9'

In [ ]:
# 对每一行数据预处理，将爬取的字符串转换为合适的数据结构

for i in range(len(DATA)):
    # process one sample
    DATA['_CPC'][i] = DATA['_CPC'][i].count(';') + 1
    DATA['_IPC'][i] = DATA['_IPC'][i].count(';') + 1
    DATA['b_cits'][i] = len(eval(DATA['b_cits'][i]))
    DATA['f_cits'][i] = eval(DATA['f_cits'][i])[0]
    aclm = DATA['_ACLM'][i]
    aclm_ls = re.findall('\n \d.', aclm)

    if len(aclm_ls) == 0:
        DATA['_ACLM'][i] = 1
    else:
        aclm_num = aclm_ls[-1]
        aclm_num = eval(aclm_num.strip('\n'))
        DATA['_ACLM'][i] = aclm_num

In [ ]:
'''
DATA: Raw data for this model.
Data: Standard data for this model.
data: A specific sample.
'''
Data = DATA
for i in range(len(Data)):
    if Data['f_cits'][i] > 4:
        Data['f_cits'][i] = 1
    else:
        Data['f_cits'][i] = 0

In [ ]:
# num = Data['f_cits'].values.sum()
# print((100-num)/num)
'''
16:84 = 1:5.25
'''

# Model: 传统结构化特征

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score


# Data.head()

In [96]:
# X:(n_samples, n_features)
X = Data.loc[:, '_CPC':'_ACLM']
del X['f_cits']

# Y: (n_samples, )
Y = Data.loc[:, 'f_cits']
Y = Y.astype('bool')
Y

0      True
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: f_cits, Length: 100, dtype: bool

In [97]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, train_size=0.8)




In [98]:
mLR = LogisticRegression()
mLR.fit(X_train, Y_train)


LogisticRegression()

In [100]:
# mLR.score(X_test, Y_test)

0.8

In [103]:
y_pred = mLR.predict(X_test)

In [110]:
print(classification_report(Y_test, y_pred))

# print(mLR.coef_, mLR.intercept_)

print(roc_auc_score(Y_test, y_pred))


              precision    recall  f1-score   support

       False       0.80      1.00      0.89        16
        True       0.00      0.00      0.00         4

    accuracy                           0.80        20
   macro avg       0.40      0.50      0.44        20
weighted avg       0.64      0.80      0.71        20

0.5


/Users/zhengao/PycharmProjects/pyAcademic/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengao/PycharmProjects/pyAcademic/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zhengao/PycharmProjects/pyAcademic/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av